#  Unit metrics for Regression Models demo

This interactive notebook guides you through go through unit metris avaliable in the ValidMind Developer Framework for regression models. It uses the [California Housing Price Prediction](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) sample dataset from Sklearn to train a simple regression model.

As part of the notebook, you will learn how to train a sample model while exploring unit tests:

- Initializing the ValidMind Developer Framework
- Loading a sample dataset provided by the library to train a simple regression model
- Running a ValidMind unit tests


## ValidMind at a glance

ValidMind's platform enables organizations to identify, document, and manage model risks for all types of models, including AI/ML models, LLMs, and statistical models. As a model developer, you use the ValidMind Developer Framework to automate documentation and validation tests, and then use the ValidMind AI Risk Platform UI to collaborate on model documentation. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind, you can make use of the following resources alongside this notebook:

- [Get started](https://docs.validmind.ai/get-started/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/developer/get-started-developer-framework.html) — The path for developers, more code samples, and our developer reference


## Before you begin

::: {.callout-tip}

### New to ValidMind?

For access to all features available in this notebook, create a free ValidMind account.

Signing up is FREE — [**Sign up now!**](https://app.prod.validmind.ai)
:::

If you encounter errors due to missing modules in your Python environment, install the modules with `pip install`, and then re-run the notebook. For more help, refer to [Installing Python Modules](https://docs.python.org/3/installing/index.html).


## Install the client library

The client library provides Python support for the ValidMind Developer Framework. To install it:


In [ ]:
%pip install -q validmind

## Initialize the client library

ValidMind generates a unique _code snippet_ for each registered model to connect with your developer environment. You initialize the client library with this code snippet, which ensures that your documentation and tests are uploaded to the correct model when you run the notebook.

Get your code snippet:

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. In the left sidebar, navigate to **Model Inventory** and click **+ Register new model**.

3. Enter the model details, making sure to select **Binary classification** as the template and **Marketing/Sales - Attrition/Churn Management** as the use case, and click **Continue**. ([Need more help?](https://docs.validmind.ai/guide/model-inventory/register-models-in-inventory.html))

4. Go to **Getting Started** and click **Copy snippet to clipboard**.

Next, replace this placeholder with your own code snippet:


In [ ]:
# Replace with your code snippet

import validmind as vm

vm.init(
    api_host="https://api.prod.validmind.ai/api/v1/tracking",
    api_key="...",
    api_secret="...",
    project="..."
)

## Initialize the Python environment

Next, let's import the necessary libraries and set up your Python environment for data analysis:


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

%matplotlib inline

## Load the sample dataset

To be able to use a sample dataset, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


## Load the sample dataset

The sample dataset used here is provided by the ValidMind library. To be able to use it, you need to import the dataset and load it into a pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html), a two-dimensional tabular data structure that makes use of rows and columns:


In [ ]:
# Import the sample dataset from the library

from validmind.datasets.regression import california_housing as demo_dataset

print(f"Loaded demo dataset with: \n\n\t• Target column: '{demo_dataset.target_column}")

raw_df = demo_dataset.load_data()
raw_df.head()

## Build the model

As part of documenting the model with the ValidMind Developer Framework, you need to preprocess the raw dataset, initialize some training and test datasets, initialize a model object you can use for testing, and then run the full suite of tests.


### Prepocess the raw dataset

Preprocessing performs a number of operations to get ready for the subsequent steps:

- Preprocess the data: Splits the DataFrame (`df`) into multiple datasets (`train_df`, `validation_df`, and `test_df`) using `demo_dataset.preprocess` to simplify preprocessing.
- Separate features and targets: Drops the target column to create feature sets (`x_train`, `x_val`) and target sets (`y_train`, `y_val`).
- Initialize RandomForestRegressor regressor: Creates an `RandomForestRegressor` object with random state set to 0.
- Set evaluation metrics: Specifies metrics for model evaluation as "errors" and "r2".
- Fit the model: Trains the model on `x_train` and `y_train` using the validation set `(x_val, y_val)`. Verbose output is disabled.


In [ ]:
train_df, validation_df, test_df = demo_dataset.preprocess(raw_df)

x_train = train_df.drop(demo_dataset.target_column, axis=1)
y_train = train_df[demo_dataset.target_column]
x_val = validation_df.drop(demo_dataset.target_column, axis=1)
y_val = validation_df[demo_dataset.target_column]

Here we create a regression model.


In [ ]:
scale = False
if scale:
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.fit_transform(x_val)

model = RandomForestRegressor(random_state=0)
model.fit(x_train, y_train)
s1 = model.score(x_train, y_train)
s2 = model.score(x_val, y_val)
print("R² of Support Vector Regressor on training set: {:.3f}".format(s1))
print("R² of Support Vector Regressor on test set: {:.3f}".format(s2))


### Initialize the ValidMind datasets

Before you can run tests, you must first initialize a ValidMind dataset object using the [`init_dataset`](https://docs.validmind.ai/validmind/validmind.html#init_dataset) function from the ValidMind (`vm`) module.

This function takes a number of arguments:

- `dataset` — the raw dataset that you want to provide as input to tests
- `input_id` - a unique identifier that allows tracking what inputs are used when running each individual test
- `target_column` — a required argument if tests require access to true values. This is the name of the target column in the dataset

With all datasets ready, you can now initialize the raw, training and test datasets (`raw_df`, `train_df` and `test_df`) created earlier into their own dataset objects using [`vm.init_dataset()`](https://docs.validmind.ai/validmind/validmind.html#init_dataset):


In [ ]:
vm_raw_dataset = vm.init_dataset(
    dataset=raw_df,
    input_id="raw_dataset",
    target_column=demo_dataset.target_column,
)

vm_train_ds = vm.init_dataset(
    dataset=train_df, input_id="train_dataset", target_column=demo_dataset.target_column
)

vm_test_ds = vm.init_dataset(
    dataset=test_df, input_id="test_dataset", target_column=demo_dataset.target_column
)

### Initialize a model object

Additionally, you need to initialize a ValidMind model objects (`vm_model`) that can be passed to other functions for analysis and tests on the data. You simply intialize this model object with [`vm.init_model()`](https://docs.validmind.ai/validmind/validmind.html#init_model):


In [ ]:
vm_model = vm.init_model(
    model,
    input_id="random_forest_regressor",
)

### Assign predictions to the datasets

We can now use the `assign_predictions()` method from the Dataset object to link existing predictions to any model. If no prediction values are passed, the method will compute predictions automatically:


In [ ]:
vm_train_ds.assign_predictions(
    model=vm_model,
)
vm_test_ds.assign_predictions(
    model=vm_model,
)

### Run tests
We are going to compute several standard evaluation metrics for regression models. These metrics are commonly used to assess the performance and accuracy of regression models.

#### Mean Absolute Error

In [ ]:
from validmind.tests import run_test

train_inputs = {"model": vm_model, "dataset": vm_train_ds}
test_inputs = {"model": vm_model, "dataset": vm_test_ds}

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.MeanAbsoluteError", test_inputs)

#### Mean Squared Error

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.MeanSquaredError", test_inputs)

#### Root Mean Squared Error

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.RootMeanSquaredError", test_inputs)

#### Mean Bias Deviation

In [ ]:
run_test("validmind.unit_metrics.regression.MeanBiasDeviation", test_inputs)

#### Mean Absolute Percentage Error

In [ ]:
run_test("validmind.unit_metrics.regression.MeanAbsolutePercentageError", inputs)

### Run model performance
We will now create a composite metric that combines multiple individual metrics into a single one, allowing us to comprehensively evaluate the performance of regression models across various error metrics.


In [ ]:
from validmind.tests import run_test

result = run_test(
    name="ModelPerformance",
    unit_metrics=[
        "validmind.unit_metrics.regression.sklearn.MeanAbsoluteError",
        "validmind.unit_metrics.regression.sklearn.MeanSquaredError",
        "validmind.unit_metrics.regression.sklearn.RootMeanSquaredError",
        "validmind.unit_metrics.regression.MeanBiasDeviation",
        "validmind.unit_metrics.regression.MeanAbsolutePercentageError"
    ],
    inputs=test_inputs,
)
result.log()

### R Squared metrics
Let's calculate the R-squared metrics, which indicate the proportion of variance in the dependent variable that is explained by the independent variables. There are two primary types of R-squared:

#### R Squared

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.RSquaredScore", test_inputs)

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.RSquaredScore", train_inputs)

#### Adjusted R Squared

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.AdjustedRSquaredScore", test_inputs)

In [ ]:
run_test("validmind.unit_metrics.regression.sklearn.AdjustedRSquaredScore", train_inputs)

### Loss metrics
Let's calculate the Huber Loss and Quantile Loss metrics, two widely utilized loss functions in regression tasks, especially when dealing with datasets containing outliers or demonstrating heteroscedasticity (varying variance levels across the data).


#### Huber loss

In [ ]:
run_test("validmind.unit_metrics.regression.HuberLoss", test_inputs)

#### Quantile loss

In [ ]:
run_test("validmind.unit_metrics.regression.QuantileLoss", test_inputs)

### Coefficients

#### Gini
Let's compute the Gini coefficient, a measure of statistical dispersion that quantifies the inequality among values in a distribution.

In [ ]:
run_test("validmind.unit_metrics.regression.GiniCoefficient", test_inputs)

#### Kolmogorov-Smirnov statistic
Let's compute the Kolmogorov-Smirnov (KS) statistic, a non-parametric test that quantifies the difference between two cumulative distribution functions (CDFs).

In [ ]:
run_test("validmind.unit_metrics.regression.KolmogorovSmirnovStatistic", test_inputs)